# NEON Foliar, Root, Litter Stoichiometry analysis

In this notebook, I will analyze foliar CNP data previously collected by NEON, alongside previously collected litter and root CN data and newly collected litter and root P data to look for patterns and links in plant-level stoichiometry across species and sites. 

### Datasets needed: 
 - P data from lab analyses, spring 2022
 - NEON database foliar data
 - NEON database litter and root CN data
 - NEON database site variables: Soil data; climate vars like MAT, MAP; species data